In [ ]:
from IPython.display import Image

In [ ]:
import os

import netCDF4 as nc
import numpy as np
# librerias graficas
import matplotlib.axes as maxes
import matplotlib.pyplot as plt
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.crs import PlateCarree
from cartopy.feature import LAND, COASTLINE, RIVERS
from cmocean.cm import thermal, speed,  algae
from mpl_toolkits.axes_grid1 import make_axes_locatable


In [ ]:
def extract_from_region(var, lat, lon, region):
    """
    extract values of a variable betweeen lat,lon coordenates.

    """
    def _order_minmax(point):
        if point[1] > point[0]:
            return point[0], point[1]
        else:
            return point[1], point[0]

    def get_index_limits(vardim, limits):
        return [np.abs(vardim - limit).argmin() for limit in limits]

    def reduce_dim(vardim, idxs):
        m, M =  _order_minmax(idxs)
        return vardim[m:M]

    def extract_var(var, lat_idxs, lon_idxs):
        lat_m, lat_M =  _order_minmax(lat_idxs)
        lon_m, lon_M =  _order_minmax(lon_idxs)
        return var[lat_m : lat_M, lon_m : lon_M]

    lat_idxs = get_index_limits(lat, region["lat"])
    lon_idxs = get_index_limits(lon, region["lon"])
    _var = extract_var(var, lat_idxs, lon_idxs)

    return _var, reduce_dim(lat, lat_idxs), reduce_dim(lon, lon_idxs)

def norm(r):
    return 100 * r / r.sum()

def long(array):
    return np.int8(np.abs(array.max() - array.min()))

def get_levels(t, step=1):
    return np.round(np.arange(t.min(), t.max() + step, step))

**Giovanni**

Utilizar Giovanni (https://giovanni.gsfc.nasa.gov/giovanni/) u otra herramienta que permita abrir imágenes de SST, Chl y Viento para luego analizarlas.



In [ ]:
Image(filename="./images/frente.png", width=700, height=None)

In [ ]:
print("Figura 1: Pagina inicial de Goivanni. Recuadrado en distintos colores se indican funcionalidades. En rojo los plots disponibles. En naranja el filtro de fechas. En amarillo el filtro de region. Estos últimos \ndos son filtros sobre disponibilidad y procesamiento de la variable elegida. En verde se recuadran las categorias de filtros sobre las variables disponibles. En gris una búsqueda por keywrord. Finalmente en\nrosa se resalta un menu de opciones, este permite resetear, plotear o ir a los resultados ya ploteados.")

In [ ]:
Image(filename="./images/type_graphs.png", width=700, height=None)

In [ ]:
print("Figura 2: Plot disponibles. En algunos casos se deben seleccionas mas de una variable")

In [ ]:
Image(filename="./images/variable-eleccion.png", width=700, height=None)

In [ ]:
print("Figura 3: Ejemplo de elección de variable. La variable tildada es la usada para el plot. Cada variable tiene dos links: el primero nos lleva a un glosario de referencia, mientras el segundo, el que esta entre parentesis, nos lleva al producto.")

In [ ]:
Image(filename="./images/plot.png", width=700, height=None)

In [ ]:
print("Figura 4: Ejemplo de plot. Recuadrado en verde historial de plots. En violeta las opciones de descarga. En rojo para volver a la selección de variables/plots.")

In [ ]:
display(Latex(r"\clearpage"))
display(Latex(r"\newpage}"))
display(Latex(r"\mbox{~}"))

**Evento de upwelling en el South Atlantic Bight, en el Océano Atlántico Noroccidental (Kyung Hoon Hyun & Ruoying H, 2010).**

El paper (Kyung Hoon Hyun & Ruoying H, 2010) trabaja en la region :  
```
{
    lat = [22, 36],
    lon = [-81, -75]
}
```

In [ ]:
STORE_DIR = "./data"

REGION = {
    "lat": [22, 36],
    "lon": [-81, -75]
}

In [ ]:
## SST 
sst_nf3 = "2004_modis_sst_monthly_night_4m.nc"
sst_nc3 =  nc.Dataset(os.path.join(STORE_DIR, sst_nf3))
sst_lat3 = sst_nc3.variables["lat"][:]
sst_lon3 = sst_nc3.variables["lon"][:]
sst_latb3 = sst_nc3.variables["lat_bnds"][:]
sst_lonb3 = sst_nc3.variables["lon_bnds"][:]
sst3 = sst_nc3.variables["MODISA_L3m_SST4_Monthly_4km_R2019_0_sst4"][:]
sst3, sst_lat3, sst_lon3 = extract_from_region(sst3, sst_lat3, sst_lon3, REGION)
sst_nc3.close()

## WIND
wind_nf = "2003_wind_stress_magnitude.nc"
wind_nc =  nc.Dataset(os.path.join(STORE_DIR, wind_nf))
var =  wind_nc.variables
wmag =  var['GSSTFM_3_SET1_INT_ST_mag'][:]
wmag_lat =  var['lat'][:]
wmag_lon =  var['lon'][:]
wmag, wmag_lat, wmag_lon = extract_from_region(wmag, wmag_lat, wmag_lon, REGION)
wind_nc.close()

wind_nf = "2003_wind_stress_vector.nc"
wind_nc =  nc.Dataset(os.path.join(STORE_DIR, wind_nf))
var =  wind_nc.variables
vel_u =  var['GSSTFM_3_SET1_INT_ST_vec_u'][:]
vel_v =  var['GSSTFM_3_SET1_INT_ST_vec_v'][:]
wvel_lat =  var['lat'][:]
wvel_lon =  var['lon'][:]
U = np.where(vel_u.data==-999, 0, vel_u.data)
V = np.where(vel_v.data==-999, 0, vel_v.data)
_vel_u, _wvel_lat, _wvel_lon = extract_from_region(U, wvel_lat, wvel_lon, REGION)
_vel_v, _wvel_lat, _wvel_lon = extract_from_region(V, wvel_lat, wvel_lon, REGION)
wind_nc.close()

grid_lon, grid_lat = np.meshgrid(_wvel_lon, _wvel_lat)

## Clorofila

ch_nf = "2003_modis_chlo_monthly_4km.nc"
ch_nc =  nc.Dataset(os.path.join(STORE_DIR, ch_nf ))
var =  ch_nc.variables
chl = var["MODISA_L3m_CHL_Mo_4km_R2022_0_chlor_a"][:]
chl_lat =  var['lat'][:]
chl_lon =  var['lon'][:]
chl, chl_lat, chl_lon = extract_from_region(chl, chl_lat, chl_lon, REGION)

ch_nc.close()

chlog_10 = np.log10(chl)
levels = [.001, .1, .2, .3, .5, .8, 1, 5, 10, 85]
log10_levels = np.log10(levels)# Tick mark positions

In [ ]:
fig, axs = plt.subplots(1, 3, subplot_kw={'projection': PlateCarree()}, layout='constrained', figsize=(15, 15), sharex=True, sharey=True)

axs[0].set_title("SST Mensual (Agosto 2003)")
axs[1].set_title("Clorofila Mensual (Agosto 2003)")
axs[2].set_title("Vientos Mensual (Agosto 2003)")
# SST
im0 = axs[0].contourf(sst_lon3, sst_lat3, sst3, levels= get_levels(sst3), vmin=24, vmax=30, cmap=thermal, zorder=0)
divider = make_axes_locatable(axs[0])
cax = divider.append_axes('bottom', size='5%', pad=0.4, axes_class=maxes.Axes)
cbar0 = fig.colorbar(im0, cax=cax, orientation='horizontal')
for label in cbar0.ax.get_xticklabels():
    label.set_rotation(45)
# CHL
im1 = axs[1].contourf(chl_lon, chl_lat,  chlog_10, levels=log10_levels, cmap=algae, vmin=np.log10(chl.min()), vmax=np.log10(chl.max()), extend='both', zorder=0)
divider = make_axes_locatable(axs[1])
cax = divider.append_axes('bottom', size='5%', pad=0.4, axes_class=maxes.Axes)
cbar1 = fig.colorbar(im1, cax=cax, orientation='horizontal')
cbar1.ax.set_xticklabels(levels)
for label in cbar1.ax.get_xticklabels():
    label.set_rotation(45)

# WIND
im2 = axs[2].contourf(wmag_lon, wmag_lat, wmag, cmap=speed, zorder=0)
divider = make_axes_locatable(axs[2])
cax = divider.append_axes('bottom', size='5%', pad=0.4, axes_class=maxes.Axes)
cbar2 = fig.colorbar(im2, cax=cax, orientation='horizontal' )
for label in cbar2.ax.get_xticklabels():
    label.set_rotation(45)

# WIND
step = 2
qv = axs[2].quiver(grid_lon[::step,::step], grid_lat[::step,::step], _vel_u[:-1, :-1][::step,::step], _vel_v[::step,::step], scale=0.5, color='k')


for i in range(3):
    axs[i].set_facecolor('white')
    axs[i].add_feature(LAND, facecolor="white", zorder=1)
    axs[i].add_feature(COASTLINE, linewidth=0.5)

    gl = axs[i].gridlines(crs=PlateCarree(), draw_labels=True, linestyle='--', zorder=0)
    gl.yformatter = LATITUDE_FORMATTER
    gl.xformatter = LONGITUDE_FORMATTER
    gl.right_labels = False
    gl.top_labels = False
plt.savefig("figurita")
plt.show()

soy unas bonitas comentarios.

In [ ]:
## SST 
sst_nf3 = "2004_modis_sst_monthly_night_4m.nc"
sst_nc3 =  nc.Dataset(os.path.join(STORE_DIR, sst_nf3))
sst_lat3 = sst_nc3.variables["lat"][:]
sst_lon3 = sst_nc3.variables["lon"][:]
sst_latb3 = sst_nc3.variables["lat_bnds"][:]
sst_lonb3 = sst_nc3.variables["lon_bnds"][:]
sst3 = sst_nc3.variables["MODISA_L3m_SST4_Monthly_4km_R2019_0_sst4"][:]
sst3, sst_lat3, sst_lon3 = extract_from_region(sst3, sst_lat3, sst_lon3, REGION)
sst_nc3.close()

## WIND
wind_nf = "2004_wind_stress_vector.nc"
wind_nc =  nc.Dataset(os.path.join(STORE_DIR, wind_nf))
var =  wind_nc.variables
vel_u =  var['GSSTFM_3_SET1_INT_ST_vec_u'][:]
vel_v =  var['GSSTFM_3_SET1_INT_ST_vec_v'][:]
wvel_lat =  var['lat'][:]
wvel_lon =  var['lon'][:]
U = np.where(vel_u.data==-999, 0, vel_u.data)
V = np.where(vel_v.data==-999, 0, vel_v.data)
_vel_u, _wvel_lat, _wvel_lon = extract_from_region(U, wvel_lat, wvel_lon, REGION)
_vel_v, _wvel_lat, _wvel_lon = extract_from_region(V, wvel_lat, wvel_lon, REGION)
wind_nc.close()

grid_lon, grid_lat = np.meshgrid(_wvel_lon, _wvel_lat)

## Clorofila

ch_nf = "2004_modis_chlo_monthly_4km.nc"
ch_nc =  nc.Dataset(os.path.join(STORE_DIR, ch_nf ))
var =  ch_nc.variables
chl = var["MODISA_L3m_CHL_Mo_4km_R2022_0_chlor_a"][:]
chl_lat =  var['lat'][:]
chl_lon =  var['lon'][:]
chl, chl_lat, chl_lon = extract_from_region(chl, chl_lat, chl_lon, REGION)

ch_nc.close()

chlog_10 = np.log10(chl)
levels = [.001, .1, .2, .3, .5, .8, 1, 5, 10, 85]
log10_levels = np.log10(levels)# Tick mark positions

In [ ]:
fig, axs = plt.subplots(1, 3, subplot_kw={'projection': PlateCarree()}, layout='constrained', figsize=(15, 15), sharex=True, sharey=True)

axs[0].set_title("SST Mensual (Agosto 2004)")
axs[1].set_title("Clorofila Mensual (Agosto 2004)")
axs[2].set_title("Vientos Mensual (Agosto 2004)")
# SST
im0 = axs[0].contourf(sst_lon3, sst_lat3, sst3, levels= get_levels(sst3), vmin=24, vmax=30, cmap=thermal, zorder=0)
divider = make_axes_locatable(axs[0])
cax = divider.append_axes('bottom', size='5%', pad=0.4, axes_class=maxes.Axes)
cbar0 = fig.colorbar(im0, cax=cax, orientation='horizontal')
for label in cbar0.ax.get_xticklabels():
    label.set_rotation(45)
# CHL
im1 = axs[1].contourf(chl_lon, chl_lat,  chlog_10, levels=log10_levels, cmap=algae, vmin=np.log10(chl.min()), vmax=np.log10(chl.max()), extend='both', zorder=0)
divider = make_axes_locatable(axs[1])
cax = divider.append_axes('bottom', size='5%', pad=0.4, axes_class=maxes.Axes)
cbar1 = fig.colorbar(im1, cax=cax, orientation='horizontal')
cbar1.ax.set_xticklabels(levels)
for label in cbar1.ax.get_xticklabels():
    label.set_rotation(45)

# WIND
step = 2
qv = axs[2].quiver(grid_lon[::step,::step], grid_lat[::step,::step], _vel_u[:-1, :-1][::step,::step], _vel_v[::step,::step], scale=0.5, color='k')

for i in range(3):
    axs[i].set_facecolor('white')
    axs[i].add_feature(LAND, facecolor="white", zorder=1)
    axs[i].add_feature(COASTLINE, linewidth=0.5)

    gl = axs[i].gridlines(crs=PlateCarree(), draw_labels=True, linestyle='--', zorder=0)
    gl.yformatter = LATITUDE_FORMATTER
    gl.xformatter = LONGITUDE_FORMATTER
    gl.right_labels = False
    gl.top_labels = False
plt.savefig("figurita")
plt.show()

soy unas bonitas  conclusiones

In [ ]:
display(Latex(r"\clearpage"))
display(Latex(r"\newpage}"))
display(Latex(r"\mbox{~}"))

**Evento de upwelling en las costas de Perú, en el Océano Pacífico (Lübbecke et al., 2020).**

En este caso, vamos a trabajar al rededor de la  region :  
```
{
    lat: [0, -25],
    lon: [-100, -70]
}
```

In [ ]:
PERU = "peru"
REGION_PERU = {
    "lat": [0, -25],
    "lon": [-100, -70]
}

In [ ]:
## SST 
sst_nf = "2005_01_SST_modis_monthly_4km.nc"
sst_nc =  nc.Dataset(os.path.join(STORE_DIR, PERU, sst_nf))
sst_lat = sst_nc.variables["lat"][:]
sst_lon = sst_nc.variables["lon"][:]
sst_latb = sst_nc.variables["lat_bnds"][:]
sst_lonb = sst_nc.variables["lon_bnds"][:]
sst = sst_nc.variables["MODISA_L3m_SST4_Monthly_4km_R2019_0_sst4"][:]
sst, sst_lat, sst_lon = extract_from_region(sst, sst_lat, sst_lon, REGION_PERU)
sst_nc.close()

## WIND
wind_nf = "2005_01_winds_magnitude.nc"
wind_nc =  nc.Dataset(os.path.join(STORE_DIR, PERU, wind_nf))
var =  wind_nc.variables
wmag =  var['GSSTFM_3_SET1_INT_ST_mag'][:]
wmag_lat =  var['lat'][:]
wmag_lon =  var['lon'][:]
wmag, wmag_lat, wmag_lon = extract_from_region(wmag, wmag_lat, wmag_lon, REGION_PERU)
wind_nc.close()

wind_nf = "2005_01_winds_vector.nc"
wind_nc =  nc.Dataset(os.path.join(STORE_DIR, PERU, wind_nf))
var =  wind_nc.variables
vel_u =  var['GSSTFM_3_SET1_INT_ST_vec_u'][:]
vel_v =  var['GSSTFM_3_SET1_INT_ST_vec_v'][:]
wvel_lat =  var['lat'][:]
wvel_lon =  var['lon'][:]
U = np.where(vel_u.data==-999, 0, vel_u.data)
V = np.where(vel_v.data==-999, 0, vel_v.data)
_vel_u, _wvel_lat, _wvel_lon = extract_from_region(U, wvel_lat, wvel_lon, REGION_PERU)
_vel_v, _wvel_lat, _wvel_lon = extract_from_region(V, wvel_lat, wvel_lon, REGION_PERU)
wind_nc.close()

grid_lon, grid_lat = np.meshgrid(wvel_lon, wvel_lat)

## Clorofila

ch_nf = "2005_01_CLH_modis_monthly_4km.nc"
ch_nc =  nc.Dataset(os.path.join(STORE_DIR, PERU, ch_nf ))
var =  ch_nc.variables
chl = var["MODISA_L3m_CHL_Mo_4km_R2022_0_chlor_a"][:]
chl_lat =  var['lat'][:]
chl_lon =  var['lon'][:]
chl, chl_lat, chl_lon = extract_from_region(chl, chl_lat, chl_lon, REGION_PERU)

ch_nc.close()

chlog_10 = np.log10(chl)
levels = [.001, .1, .2, .3, .5, .8, 1, 5, 10, 85]
log10_levels = np.log10(levels)# Tick mark positions

In [ ]:
fig, axs = plt.subplots(1, 2, subplot_kw={'projection': PlateCarree()}, layout='constrained', figsize=(15, 30), sharex=True, sharey=True)

axs[0].set_title("SST Mensual (Enero 2005)")
axs[1].set_title("Clorofila Mensual (Enero 2005)")

# SST
im0 = axs[0].contourf(sst_lon, sst_lat, sst, levels= get_levels(sst), vmin=15, vmax=30, cmap=thermal, zorder=0)
divider = make_axes_locatable(axs[0])
cax = divider.append_axes('bottom', size='5%', pad=0.4, axes_class=maxes.Axes)
cbar0 = fig.colorbar(im0, cax=cax, orientation='horizontal')
for label in cbar0.ax.get_xticklabels():
    label.set_rotation(45)
# CHL
im1 = axs[1].contourf(chl_lon, chl_lat,  chlog_10, levels=log10_levels, cmap=algae, vmin=np.log10(chl.min()), vmax=np.log10(chl.max()), extend='both', zorder=0)
divider = make_axes_locatable(axs[1])
cax = divider.append_axes('bottom', size='5%', pad=0.4, axes_class=maxes.Axes)
cbar1 = fig.colorbar(im1, cax=cax, orientation='horizontal')
cbar1.ax.set_xticklabels(levels)
for label in cbar1.ax.get_xticklabels():
    label.set_rotation(45)


for i in range(2):
    axs[i].set_facecolor('white')
    axs[i].add_feature(LAND, facecolor="white", zorder=1)
    axs[i].add_feature(COASTLINE, linewidth=0.5)

    gl = axs[i].gridlines(crs=PlateCarree(), draw_labels=True, linestyle='--', zorder=0)
    gl.yformatter = LATITUDE_FORMATTER
    gl.xformatter = LONGITUDE_FORMATTER
    gl.right_labels = False
    gl.top_labels = False
plt.show()

In [ ]:
step = 10

fig, axs = plt.subplots(1, 1, subplot_kw={'projection': PlateCarree()}, layout='constrained', figsize=(8, 8), sharex=True, sharey=True)
axs.set_title("Vientos mensual (Enero 2005)")
im2 = axs.contourf(wmag_lon, wmag_lat, wmag, cmap=speed, zorder=0)
divider = make_axes_locatable(axs)
cax = divider.append_axes('bottom', size='5%', pad=0.4, axes_class=maxes.Axes)
cbar2 = fig.colorbar(im2, cax=cax, orientation='horizontal' )
for label in cbar2.ax.get_xticklabels():
    label.set_rotation(45)
    
qv = axs.quiver(_wvel_lon[::step], _wvel_lat[::step], _vel_u[:-1, :-1][::step,::step], _vel_v[::step,::step], scale=2, color='k')


axs.set_facecolor('white')
axs.add_feature(LAND, facecolor="white", zorder=1)
axs.add_feature(COASTLINE, linewidth=0.5)

gl = axs.gridlines(crs=PlateCarree(), draw_labels=True, linestyle='--', zorder=0)
gl.yformatter = LATITUDE_FORMATTER
gl.xformatter = LONGITUDE_FORMATTER
gl.right_labels = False
gl.top_labels = False
plt.savefig("winds")
plt.show()

In [ ]:
display(Latex(r"\mbox{~}"))

In [ ]:
## SST 
sst_nf = "2005_04_SST_modis_monthly_4km.nc"
sst_nc =  nc.Dataset(os.path.join(STORE_DIR, PERU, sst_nf))
sst_lat = sst_nc.variables["lat"][:]
sst_lon = sst_nc.variables["lon"][:]
sst_latb = sst_nc.variables["lat_bnds"][:]
sst_lonb = sst_nc.variables["lon_bnds"][:]
sst = sst_nc.variables["MODISA_L3m_SST4_Monthly_4km_R2019_0_sst4"][:]
sst, sst_lat, sst_lon = extract_from_region(sst, sst_lat, sst_lon, REGION_PERU)
sst_nc.close()

## WIND
wind_nf = "2005_04_winds_magnitude.nc"
wind_nc =  nc.Dataset(os.path.join(STORE_DIR, PERU, wind_nf))
var =  wind_nc.variables
wmag =  var['GSSTFM_3_SET1_INT_ST_mag'][:]
wmag_lat =  var['lat'][:]
wmag_lon =  var['lon'][:]
wmag, wmag_lat, wmag_lon = extract_from_region(wmag, wmag_lat, wmag_lon, REGION_PERU)
wind_nc.close()

wind_nf = "2005_04_winds_vector.nc"
wind_nc =  nc.Dataset(os.path.join(STORE_DIR, PERU, wind_nf))
var =  wind_nc.variables
vel_u =  var['GSSTFM_3_SET1_INT_ST_vec_u'][:]
vel_v =  var['GSSTFM_3_SET1_INT_ST_vec_v'][:]
wvel_lat =  var['lat'][:]
wvel_lon =  var['lon'][:]
U = np.where(vel_u.data==-999, 0, vel_u.data)
V = np.where(vel_v.data==-999, 0, vel_v.data)
_vel_u, _wvel_lat, _wvel_lon = extract_from_region(U, wvel_lat, wvel_lon, REGION_PERU)
_vel_v, _wvel_lat, _wvel_lon = extract_from_region(V, wvel_lat, wvel_lon, REGION_PERU)
wind_nc.close()

grid_lon, grid_lat = np.meshgrid(wvel_lon, wvel_lat)

## Clorofila

ch_nf = "2005_04_CLH_modis_monthly_4km.nc"
ch_nc =  nc.Dataset(os.path.join(STORE_DIR, PERU, ch_nf ))
var =  ch_nc.variables
chl = var["MODISA_L3m_CHL_Mo_4km_R2022_0_chlor_a"][:]
chl_lat =  var['lat'][:]
chl_lon =  var['lon'][:]
chl, chl_lat, chl_lon = extract_from_region(chl, chl_lat, chl_lon, REGION_PERU)

ch_nc.close()

chlog_10 = np.log10(chl)
levels = [.001, .1, .2, .3, .5, .8, 1, 5, 10, 85]
log10_levels = np.log10(levels)# Tick mark positions

In [ ]:
fig, axs = plt.subplots(1, 2, subplot_kw={'projection': PlateCarree()}, layout='constrained', figsize=(15, 15), sharex=True, sharey=True)

axs[0].set_title("SST Mensual (Abril 2005)")
axs[1].set_title("Clorofila Mensual (Abril 2005)")

# SST
im0 = axs[0].contourf(sst_lon, sst_lat, sst, levels= get_levels(sst), vmin=15, vmax=30, cmap=thermal, zorder=0)
divider = make_axes_locatable(axs[0])
cax = divider.append_axes('bottom', size='5%', pad=0.4, axes_class=maxes.Axes)
cbar0 = fig.colorbar(im0, cax=cax, orientation='horizontal')
for label in cbar0.ax.get_xticklabels():
    label.set_rotation(45)
# CHL
im1 = axs[1].contourf(chl_lon, chl_lat,  chlog_10, levels=log10_levels, cmap=algae, vmin=np.log10(chl.min()), vmax=np.log10(chl.max()), extend='both', zorder=0)
divider = make_axes_locatable(axs[1])
cax = divider.append_axes('bottom', size='5%', pad=0.4, axes_class=maxes.Axes)
cbar1 = fig.colorbar(im1, cax=cax, orientation='horizontal')
cbar1.ax.set_xticklabels(levels)
for label in cbar1.ax.get_xticklabels():
    label.set_rotation(45)

for i in range(2):
    axs[i].set_facecolor('white')
    axs[i].add_feature(LAND, facecolor="white", zorder=1)
    axs[i].add_feature(COASTLINE, linewidth=0.5)

    gl = axs[i].gridlines(crs=PlateCarree(), draw_labels=True, linestyle='--', zorder=0)
    gl.yformatter = LATITUDE_FORMATTER
    gl.xformatter = LONGITUDE_FORMATTER
    gl.right_labels = False
    gl.top_labels = False
plt.show()

In [ ]:
step = 5

fig, axs = plt.subplots(1, 1, subplot_kw={'projection': PlateCarree()}, layout='constrained', figsize=(8, 8), sharex=True, sharey=True)
axs.set_title("Vientos mensual (Abril 2005)")
im2 = axs.contourf(wmag_lon, wmag_lat, wmag, cmap=speed, zorder=0)
divider = make_axes_locatable(axs)
cax = divider.append_axes('bottom', size='5%', pad=0.4, axes_class=maxes.Axes)
cbar2 = fig.colorbar(im2, cax=cax, orientation='horizontal' )
for label in cbar2.ax.get_xticklabels():
    label.set_rotation(45)
    
#qv = axs.quiver(grid_lon[::step,::step], grid_lat[::step,::step], vel_u[:-1, :-1][::step,::step], vel_v[::step,::step], scale=900, color='k')
qv = axs.quiver(_wvel_lon[::step], _wvel_lat[::step], _vel_u[:-1, :-1][::step,::step], _vel_v[::step,::step], scale=2, color='k')
axs.set_facecolor('white')
axs.add_feature(LAND, facecolor="white", zorder=1)
axs.add_feature(COASTLINE, linewidth=0.5)

gl = axs.gridlines(crs=PlateCarree(), draw_labels=True, linestyle='--', zorder=0)
gl.yformatter = LATITUDE_FORMATTER
gl.xformatter = LONGITUDE_FORMATTER
gl.right_labels = False
gl.top_labels = False
plt.show()
plt.show()